## Test environment

In [162]:
import primer3
import pandas as pd
import sqlite3
from datetime import datetime

# Import modules
from multiplexdesigner.utils import MultiplexPanel

In [163]:
def create_primer_database(data, db_name='primer_results.db'):
    """
    Convert primer design results to a SQLite database with proper schema
    """
    
    # Connect to SQLite database (creates if doesn't exist)
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    # Create tables with proper schema
    create_tables(cursor)
    
    # Insert the data
    analysis_id = insert_analysis_summary(cursor, data)
    insert_primer_pairs(cursor, data, analysis_id)
    insert_primers(cursor, data, analysis_id)
    insert_internal_primers(cursor, data, analysis_id)
    
    # Commit changes and close
    conn.commit()
    conn.close()
    
    print(f"Database created successfully: {db_name}")
    return db_name

def create_tables(cursor):
    """Create all necessary tables with proper relationships"""
    
    # Analysis summary table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS analysis_summary (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            created_date TEXT DEFAULT CURRENT_TIMESTAMP,
            left_primers_considered INTEGER,
            right_primers_considered INTEGER,
            internal_primers_considered INTEGER,
            pairs_considered INTEGER,
            left_primers_returned INTEGER,
            right_primers_returned INTEGER,
            internal_primers_returned INTEGER,
            pairs_returned INTEGER,
            left_explain TEXT,
            right_explain TEXT,
            internal_explain TEXT,
            pair_explain TEXT
        )
    ''')
    
    # Primer pairs table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS primer_pairs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            analysis_id INTEGER,
            pair_rank INTEGER,
            penalty REAL,
            compl_any_th REAL,
            compl_end_th REAL,
            product_size INTEGER,
            product_tm REAL,
            left_primer_id INTEGER,
            right_primer_id INTEGER,
            internal_primer_id INTEGER,
            FOREIGN KEY (analysis_id) REFERENCES analysis_summary (id)
        )
    ''')
    
    # Left primers table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS left_primers (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            analysis_id INTEGER,
            primer_rank INTEGER,
            sequence TEXT,
            start_pos INTEGER,
            length INTEGER,
            penalty REAL,
            tm REAL,
            gc_percent REAL,
            self_any_th REAL,
            self_end_th REAL,
            hairpin_th REAL,
            end_stability REAL,
            FOREIGN KEY (analysis_id) REFERENCES analysis_summary (id)
        )
    ''')
    
    # Right primers table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS right_primers (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            analysis_id INTEGER,
            primer_rank INTEGER,
            sequence TEXT,
            start_pos INTEGER,
            length INTEGER,
            penalty REAL,
            tm REAL,
            gc_percent REAL,
            self_any_th REAL,
            self_end_th REAL,
            hairpin_th REAL,
            end_stability REAL,
            FOREIGN KEY (analysis_id) REFERENCES analysis_summary (id)
        )
    ''')
    
    # Internal primers table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS internal_primers (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            analysis_id INTEGER,
            primer_rank INTEGER,
            sequence TEXT,
            start_pos INTEGER,
            length INTEGER,
            penalty REAL,
            tm REAL,
            gc_percent REAL,
            self_any_th REAL,
            self_end_th REAL,
            hairpin_th REAL,
            FOREIGN KEY (analysis_id) REFERENCES analysis_summary (id)
        )
    ''')

def insert_analysis_summary(cursor, data):
    """Insert analysis summary and return the analysis_id"""
    cursor.execute('''
        INSERT INTO analysis_summary (
            left_primers_returned, right_primers_returned, 
            internal_primers_returned, pairs_returned,
            left_explain, right_explain, internal_explain, pair_explain
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        data.get('PRIMER_LEFT_NUM_RETURNED', 0),
        data.get('PRIMER_RIGHT_NUM_RETURNED', 0),
        data.get('PRIMER_INTERNAL_NUM_RETURNED', 0),
        data.get('PRIMER_PAIR_NUM_RETURNED', 0),
        data.get('PRIMER_LEFT_EXPLAIN', ''),
        data.get('PRIMER_RIGHT_EXPLAIN', ''),
        data.get('PRIMER_INTERNAL_EXPLAIN', ''),
        data.get('PRIMER_PAIR_EXPLAIN', '')
    ))
    return cursor.lastrowid

def insert_primer_pairs(cursor, data, analysis_id):
    """Insert primer pair data"""
    pairs = data.get('PRIMER_PAIR', [])
    
    for rank, pair in enumerate(pairs):
        cursor.execute('''
            INSERT INTO primer_pairs (
                analysis_id, pair_rank, penalty, compl_any_th, compl_end_th,
                product_size, product_tm
            ) VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', (
            analysis_id, rank, 
            pair.get('PENALTY', 0),
            pair.get('COMPL_ANY_TH', 0),
            pair.get('COMPL_END_TH', 0),
            pair.get('PRODUCT_SIZE', 0),
            pair.get('PRODUCT_TM', 0)
        ))

def insert_primers(cursor, data, analysis_id):
    """Insert left and right primer data"""
    
    # Left primers
    left_primers = data.get('PRIMER_LEFT', [])
    for rank, primer in enumerate(left_primers):
        coords = primer.get('COORDS', [0, 0])
        cursor.execute('''
            INSERT INTO left_primers (
                analysis_id, primer_rank, sequence, start_pos, length,
                penalty, tm, gc_percent, self_any_th, self_end_th,
                hairpin_th, end_stability
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            analysis_id, rank,
            primer.get('SEQUENCE', ''),
            coords[0] if len(coords) > 0 else 0,
            coords[1] if len(coords) > 1 else 0,
            primer.get('PENALTY', 0),
            primer.get('TM', 0),
            primer.get('GC_PERCENT', 0),
            primer.get('SELF_ANY_TH', 0),
            primer.get('SELF_END_TH', 0),
            primer.get('HAIRPIN_TH', 0),
            primer.get('END_STABILITY', 0)
        ))
    
    # Right primers
    right_primers = data.get('PRIMER_RIGHT', [])
    for rank, primer in enumerate(right_primers):
        coords = primer.get('COORDS', [0, 0])
        cursor.execute('''
            INSERT INTO right_primers (
                analysis_id, primer_rank, sequence, start_pos, length,
                penalty, tm, gc_percent, self_any_th, self_end_th,
                hairpin_th, end_stability
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            analysis_id, rank,
            primer.get('SEQUENCE', ''),
            coords[0] if len(coords) > 0 else 0,
            coords[1] if len(coords) > 1 else 0,
            primer.get('PENALTY', 0),
            primer.get('TM', 0),
            primer.get('GC_PERCENT', 0),
            primer.get('SELF_ANY_TH', 0),
            primer.get('SELF_END_TH', 0),
            primer.get('HAIRPIN_TH', 0),
            primer.get('END_STABILITY', 0)
        ))

def insert_internal_primers(cursor, data, analysis_id):
    """Insert internal primer data"""
    internal_primers = data.get('PRIMER_INTERNAL', [])
    
    for rank, primer in enumerate(internal_primers):
        coords = primer.get('COORDS', [0, 0])
        cursor.execute('''
            INSERT INTO internal_primers (
                analysis_id, primer_rank, sequence, start_pos, length,
                penalty, tm, gc_percent, self_any_th, self_end_th, hairpin_th
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            analysis_id, rank,
            primer.get('SEQUENCE', ''),
            coords[0] if len(coords) > 0 else 0,
            coords[1] if len(coords) > 1 else 0,
            primer.get('PENALTY', 0),
            primer.get('TM', 0),
            primer.get('GC_PERCENT', 0),
            primer.get('SELF_ANY_TH', 0),
            primer.get('SELF_END_TH', 0),
            primer.get('HAIRPIN_TH', 0)
        ))

def query_database(db_name='primer_results.db'):
    """Example queries to retrieve data from the database"""
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    print("=== DATABASE QUERIES ===\n")
    
    # Query 1: Best primer pairs
    print("1. Best Primer Pairs (ordered by penalty):")
    cursor.execute('''
        SELECT pair_rank, penalty, product_size, product_tm, compl_any_th, compl_end_th
        FROM primer_pairs 
        ORDER BY penalty 
        LIMIT 5
    ''')
    
    for row in cursor.fetchall():
        print(f"   Rank {row[0]}: Penalty={row[1]:.3f}, Size={row[2]}bp, Tm={row[3]:.1f}°C")
    
    # Query 2: All primer sequences for best pair
    print("\n2. Primer Sequences for Best Pair:")
    cursor.execute('''
        SELECT l.sequence as left_seq, r.sequence as right_seq, i.sequence as internal_seq
        FROM primer_pairs pp
        JOIN left_primers l ON l.primer_rank = 0 AND l.analysis_id = pp.analysis_id
        JOIN right_primers r ON r.primer_rank = 0 AND r.analysis_id = pp.analysis_id  
        JOIN internal_primers i ON i.primer_rank = 0 AND i.analysis_id = pp.analysis_id
        WHERE pp.pair_rank = 0
    ''')
    
    result = cursor.fetchone()
    if result:
        print(f"   Left:     {result[0]}")
        print(f"   Right:    {result[1]}")
        print(f"   Internal: {result[2]}")
    
    # Query 3: Analysis summary
    print("\n3. Analysis Summary:")
    cursor.execute('SELECT * FROM analysis_summary ORDER BY id DESC LIMIT 1')
    summary = cursor.fetchone()
    if summary:
        print(f"   Left primers found: {summary[5]}")
        print(f"   Right primers found: {summary[6]}")
        print(f"   Primer pairs found: {summary[8]}")
    
    conn.close()

# Convert different sections to DataFrames
def convert_primer_data_to_tables(data):
    tables = {}
    
    # 1. Primer Pairs Summary
    if 'PRIMER_PAIR' in data:
        primer_pairs_df = pd.DataFrame(data['PRIMER_PAIR'])
        primer_pairs_df.index.name = 'Pair_ID'
        primer_pairs_df = primer_pairs_df.round(3)  # Round decimals
        tables['primer_pairs'] = primer_pairs_df
    
    # 2. Left Primers
    if 'PRIMER_LEFT' in data:
        left_primers_df = pd.DataFrame(data['PRIMER_LEFT'])
        left_primers_df.index.name = 'Left_Primer_ID'
        left_primers_df = left_primers_df.round(3)
        tables['left_primers'] = left_primers_df
    
    # 3. Right Primers  
    if 'PRIMER_RIGHT' in data:
        right_primers_df = pd.DataFrame(data['PRIMER_RIGHT'])
        right_primers_df.index.name = 'Right_Primer_ID'
        right_primers_df = right_primers_df.round(3)
        tables['right_primers'] = right_primers_df
    
    # 4. Internal Primers
    if 'PRIMER_INTERNAL' in data:
        internal_primers_df = pd.DataFrame(data['PRIMER_INTERNAL'])
        internal_primers_df.index.name = 'Internal_Primer_ID'
        internal_primers_df = internal_primers_df.round(3)
        tables['internal_primers'] = internal_primers_df
    
    # 5. Summary Statistics
    summary_data = {
        'Metric': ['Left Primers Returned', 'Right Primers Returned', 'Internal Primers Returned', 'Primer Pairs Returned'],
        'Count': [
            data.get('PRIMER_LEFT_NUM_RETURNED', 0),
            data.get('PRIMER_RIGHT_NUM_RETURNED', 0), 
            data.get('PRIMER_INTERNAL_NUM_RETURNED', 0),
            data.get('PRIMER_PAIR_NUM_RETURNED', 0)
        ]
    }
    tables['summary'] = pd.DataFrame(summary_data)
    
    return tables

def create_primer_dataframe(primer_data):
    """
    Convert primer design results to organized pandas DataFrames
    """
    
    # Method 1: Create a comprehensive primer pairs DataFrame
    primer_pairs = []
    
    num_pairs = primer_data.get('PRIMER_PAIR_NUM_RETURNED', 0)
    
    for i in range(num_pairs):
        pair_info = {
            'pair_id': i,
            'pair_penalty': primer_data.get(f'PRIMER_PAIR_{i}_PENALTY'),
            'product_size': primer_data.get(f'PRIMER_PAIR_{i}_PRODUCT_SIZE'),
            'product_tm': primer_data.get(f'PRIMER_PAIR_{i}_PRODUCT_TM'),
            'compl_any_th': primer_data.get(f'PRIMER_PAIR_{i}_COMPL_ANY_TH'),
            'compl_end_th': primer_data.get(f'PRIMER_PAIR_{i}_COMPL_END_TH'),
            
            # Left primer info
            'left_sequence': primer_data.get(f'PRIMER_LEFT_{i}_SEQUENCE'),
            'left_coords': primer_data.get(f'PRIMER_LEFT_{i}'),
            'left_tm': primer_data.get(f'PRIMER_LEFT_{i}_TM'),
            'left_gc_percent': primer_data.get(f'PRIMER_LEFT_{i}_GC_PERCENT'),
            'left_penalty': primer_data.get(f'PRIMER_LEFT_{i}_PENALTY'),
            'left_self_any_th': primer_data.get(f'PRIMER_LEFT_{i}_SELF_ANY_TH'),
            'left_self_end_th': primer_data.get(f'PRIMER_LEFT_{i}_SELF_END_TH'),
            'left_hairpin_th': primer_data.get(f'PRIMER_LEFT_{i}_HAIRPIN_TH'),
            'left_end_stability': primer_data.get(f'PRIMER_LEFT_{i}_END_STABILITY'),
            
            # Right primer info
            'right_sequence': primer_data.get(f'PRIMER_RIGHT_{i}_SEQUENCE'),
            'right_coords': primer_data.get(f'PRIMER_RIGHT_{i}'),
            'right_tm': primer_data.get(f'PRIMER_RIGHT_{i}_TM'),
            'right_gc_percent': primer_data.get(f'PRIMER_RIGHT_{i}_GC_PERCENT'),
            'right_penalty': primer_data.get(f'PRIMER_RIGHT_{i}_PENALTY'),
            'right_self_any_th': primer_data.get(f'PRIMER_RIGHT_{i}_SELF_ANY_TH'),
            'right_self_end_th': primer_data.get(f'PRIMER_RIGHT_{i}_SELF_END_TH'),
            'right_hairpin_th': primer_data.get(f'PRIMER_RIGHT_{i}_HAIRPIN_TH'),
            'right_end_stability': primer_data.get(f'PRIMER_RIGHT_{i}_END_STABILITY'),
        }
        primer_pairs.append(pair_info)
    
    df_pairs = pd.DataFrame(primer_pairs)

    return(df_pairs)


In [164]:
# Get the mutation list (junctions) around which to design primer pairs
panel = MultiplexPanel("test_panel", "hg38")
panel.load_config(config_path="./config/designer_default_config.json")
panel.import_junctions_csv("./data/design_regions.csv")

Successfully imported 4 junctions from ./data/design_regions.csv


In [165]:
print(panel.design_config['primer_length_range'])

[15, 30]


In [166]:
print(panel.junctions)

[Junction(EGFR_T790M, chr7:55181378-55181378), Junction(KRAS_G12D, chr12:25245350-25245350), Junction(KRAS_G13R, chr12:25245348-25245349), Junction(BRAF_V600E, chr7:140753336-140753336)]


In [167]:
panel.merge_close_junctions()

Merging junctions within 100 bp on same chromosome...
Merged 2 junctions: KRAS_G13R_KRAS_G12D
After merging: 3 junctions remain


In [168]:
print(panel.junctions)

[Junction(KRAS_G13R_KRAS_G12D, chr12:25245348-25245350), Junction(EGFR_T790M, chr7:55181378-55181378), Junction(BRAF_V600E, chr7:140753336-140753336)]


In [169]:
fasta_file = '/Users/ctosimsen/hg38/hg38.fa'
panel.extract_design_regions_from_fasta(fasta_file, padding=200)

Extracting design regions from /Users/ctosimsen/hg38/hg38.fa with 200bp padding...
Successfully extracted 3 design regions


In [170]:
for junction in panel.junctions:
    if hasattr(junction, 'design_region'):
        print(f"\n{junction.name}:")
        print(f"  Region: {junction.design_region} ")



KRAS_G13R_KRAS_G12D:
  Region: TTCAGATAACTTAACTTTCAGCATAATTATCTTGTAATAAGTACTCATGAAAATGGTCAGAGAAACCTTTATCTGTATCAAAGAATGGTCCTGCACCAGTAATATGCATATTAAAACAAGATTTACCTCTATTGTTGGATCATATTCGTCCACAAAATGATTCTGAATTAGCTGTATCGTCAAGGCACTCTTGCCTACGCCACCAGCTCCAACTACCACAAGTTTATATTCAGTCATTTTCAGCAGGCCTTATAATAAAAATAATGAAAATGTGACTATATTAGAACATGTCACACATAAGGTTAATACACTATCAAATACTCCACCAGTACCTTTTAATACAAACTCACCTTTATATGAAAAATTATTTCAAAATACCTTACAAAATTCAATCATGAAAATT 

EGFR_T790M:
  Region: TACGTATTTTGAAACTCAAGATCGCATTCATGCGTCTTCACCTGGAAGGGGTCCATGTGCCCCTCCTTCTGGCCACCATGCGAAGCCACACTGACGTGCCTCTCCCTCCCTCCAGGAAGCCTACGTGATGGCCAGCGTGGACAACCCCCACGTGTGCCGCCTGCTGGGCATCTGCCTCACCTCCACCGTGCAGCTCATCACGCAGCTCATGCCCTTCGGCTGCCTCCTGGACTATGTCCGGGAACACAAAGACAATATTGGCTCCCAGTACCTGCTCAACTGGTGTGTGCAGATCGCAAAGGTAATCAGGGAAGGGAGATACGGGGAGGGGAGATAAGGAGCCAGGATCCTCACATGCGGTCTGCGCTCCTGGGATAGCAAGAGTTTGCCATGGGGATATG 

BRAF_V600E:
  Region: AAAAAATAAGAACACTGATTTTTGTGAATACTGGGAACTATGAAAATACTATAGTTGAGACCTTCAATGACTTTCTAGTAACTCAGCAGCATCTCAGGGCCAAAAATTTAATCA

In [171]:
print(vars(panel.junctions[0]))

{'name': 'KRAS_G13R_KRAS_G12D', 'chrom': 'chr12', 'five_prime': 25245348, 'three_prime': 25245350, 'design_region': 'TTCAGATAACTTAACTTTCAGCATAATTATCTTGTAATAAGTACTCATGAAAATGGTCAGAGAAACCTTTATCTGTATCAAAGAATGGTCCTGCACCAGTAATATGCATATTAAAACAAGATTTACCTCTATTGTTGGATCATATTCGTCCACAAAATGATTCTGAATTAGCTGTATCGTCAAGGCACTCTTGCCTACGCCACCAGCTCCAACTACCACAAGTTTATATTCAGTCATTTTCAGCAGGCCTTATAATAAAAATAATGAAAATGTGACTATATTAGAACATGTCACACATAAGGTTAATACACTATCAAATACTCCACCAGTACCTTTTAATACAAACTCACCTTTATATGAAAAATTATTTCAAAATACCTTACAAAATTCAATCATGAAAATT', 'design_start': 25245148, 'design_end': 25245550, 'junction_length': None, 'jmin_coordinate': None, 'jmax_coordinate': None}


In [172]:
panel.calculate_junction_coordinates_in_design_region()

Calculated junction coordinates for 3 junctions


In [173]:
print(vars(panel.junctions[0]))

{'name': 'KRAS_G13R_KRAS_G12D', 'chrom': 'chr12', 'five_prime': 25245348, 'three_prime': 25245350, 'design_region': 'TTCAGATAACTTAACTTTCAGCATAATTATCTTGTAATAAGTACTCATGAAAATGGTCAGAGAAACCTTTATCTGTATCAAAGAATGGTCCTGCACCAGTAATATGCATATTAAAACAAGATTTACCTCTATTGTTGGATCATATTCGTCCACAAAATGATTCTGAATTAGCTGTATCGTCAAGGCACTCTTGCCTACGCCACCAGCTCCAACTACCACAAGTTTATATTCAGTCATTTTCAGCAGGCCTTATAATAAAAATAATGAAAATGTGACTATATTAGAACATGTCACACATAAGGTTAATACACTATCAAATACTCCACCAGTACCTTTTAATACAAACTCACCTTTATATGAAAAATTATTTCAAAATACCTTACAAAATTCAATCATGAAAATT', 'design_start': 25245148, 'design_end': 25245550, 'junction_length': 403, 'jmin_coordinate': 196, 'jmax_coordinate': 204}


In [174]:
panel.design_config

{'genome': 'hg38',
 'primer_optimal_length': 20,
 'primer_length_range': [15, 30],
 'amplicon_gap_range': [20, 60],
 'max_amplicon_length': 100,
 'junction_padding_bases': 3,
 'initial_solutions': 100,
 'top_solutions_to_keep': 4,
 'variant_threshold': 0.01,
 '5_primer_tail': '',
 '3_prime_tail': '',
 'snp_penalty': 1.0,
 'primer_length_penalty': 1.0,
 'primer_complexity_penalty': 1.0,
 'polyA_penalty': 5,
 'polyT_penalty': 5,
 'polyC_penalty': 10,
 'polyG_penalty': 10,
 'amplicon_length_penalty': 1.0,
 'accepted_gc_range': [20.0, 80.0],
 'primer_pair_max_tm_difference': 3.0}

In [175]:
panel.pcr_config

{'annealing_temperature': 60,
 'opt_melting_temperature': 60,
 'tm_difference': 3.0,
 'primer_concentration': 50,
 'dntp_concentration': 0.8,
 'dna_concentration': 50,
 'mv_concentration': 50,
 'dv_concentration': 1.5,
 'dmso_concentration': 0.0,
 'dmso_fact': 0.6,
 'formamide_concentration': 0.8}

In [ ]:
min_product_length = 2 * panel.design_config['primer_length_range'][0] + junction.jmax_coordinate - junction.jmin_coordinate
max_product_length = panel.design_config['max_amplicon_length']


# Set global design arguments
global_args={
    'PRIMER_TASK': "pick_sequencing_primers",
    'PRIMER_OPT_SIZE': panel.design_config['primer_optimal_length'],
    'PRIMER_WT_SIZE_LT': panel.design_config['primer_length_penalty'],       # Penalty weight for primers shorter than PRIMER_OPT_SIZE.
    'PRIMER_WT_SIZE_GT': panel.design_config['primer_length_penalty'],       # Penalty weight for primers longer than PRIMER_OPT_SIZE.
    'PRIMER_MIN_SIZE': panel.design_config['primer_length_range'][0],
    'PRIMER_MAX_SIZE': panel.design_config['primer_length_range'][1],
    'PRIMER_OPT_TM': panel.pcr_config['opt_melting_temperature'],
    'PRIMER_TM_FORMULA': 1,
    'PRIMER_SALT_CORRECTIONS': 1,
    'PRIMER_MIN_TM': panel.pcr_config['opt_melting_temperature']-panel.pcr_config['tm_difference'],
    'PRIMER_MAX_TM': panel.pcr_config['opt_melting_temperature']+panel.pcr_config['tm_difference'],
    'PRIMER_SALT_MONOVALENT': panel.pcr_config['mv_concentration'],           # The millimolar (mM) concentration of monovalent salt cations (usually KCl) in the PCR.
    'PRIMER_SALT_DIVALENT': panel.pcr_config['dv_concentration'],             # The millimolar concentration of divalent salt cations (usually MgCl^(2+)) in the PCR. 
    'PRIMER_DNA_CONC': panel.pcr_config['primer_concentration'],
    'PRIMER_DNTP_CONC': panel.pcr_config['dntp_concentration'],               # Millimolar concentration of the sum of all deoxyribonucleotide triphosphates (e.g. 4x 0.2=0.8)
    'PRIMER_DMSO_CONC': panel.pcr_config['dmso_concentration'],
    'PRIMER_FORMAMIDE_CONC': panel.pcr_config['formamide_concentration'],
    'PRIMER_NUM_RETURN': 10,                                                   # The maximum number of primer (pairs) to return.
    'PRIMER_OPT_GC_PERCENT': 50.0,                                             # Optimum GC percent. This parameter influences primer selection only if PRIMER_WT_GC_PERCENT_GT or PRIMER_WT_GC_PERCENT_LT are non-0.
    'PRIMER_WT_GC_PERCENT_LT': 0.0,
    'PRIMER_WT_GC_PERCENT_GT': 0.0,
    'PRIMER_MIN_GC': panel.design_config['accepted_gc_range'][0],
    'PRIMER_MAX_GC': panel.design_config['accepted_gc_range'][1],
    'PRIMER_MAX_POLY_X': panel.design_config['max_poly_x'],
    'PRIMER_MAX_NS_ACCEPTED': 0,
    'PRIMER_PAIR_MAX_DIFF_TM': panel.design_config['primer_pair_max_tm_difference'],
    'PRIMER_MAX_SELF_ANY': 8.0,
    'PRIMER_MAX_SELF_END': 3.0,
    'PRIMER_PAIR_MAX_COMPL_ANY': 8.0,
    'PRIMER_PAIR_MAX_COMPL_END': 8.0,
    'PRIMER_MAX_SELF_ANY_TH': 45.0,
    'PRIMER_MAX_SELF_END_TH': 35.0,
    'PRIMER_PAIR_MAX_COMPL_ANY_TH': 45.0,
    'PRIMER_PAIR_MAX_COMPL_END_TH': 35.0,
    'PRIMER_MAX_HAIRPIN_TH': 24.0,
    'PRIMER_MAX_TEMPLATE_MISPRIMING_TH': 35.0,
    'PRIMER_PAIR_MAX_TEMPLATE_MISPRIMING_TH': 47.0,
    'PRIMER_WT_SELF_ANY': 5.0,
    'PRIMER_WT_SELF_ANY_TH': 5.0,
    'PRIMER_WT_HAIRPIN_TH': 1.0,
    'PRIMER_WT_TEMPLATE_MISPRIMING_TH': 1.0,
    'PRIMER_PAIR_WT_COMPL_ANY': 5.0,
    'PRIMER_PAIR_WT_COMPL_ANY_TH': 5.0,
    'PRIMER_PAIR_WT_COMPL_END_TH': 5.0,
    'PRIMER_PAIR_WT_TEMPLATE_MISPRIMING_TH': 1.0,
    'PRIMER_THERMODYNAMIC_OLIGO_ALIGNMENT': 1,
    'PRIMER_THERMODYNAMIC_TEMPLATE_ALIGNMENT': 1,
    'PRIMER_PRODUCT_SIZE_RANGE': [[min_product_length, max_product_length]]
}


In [186]:
results_list = []


for junction in panel.junctions:
    # Set the sequence arguments
    seq_args={
        'SEQUENCE_ID': junction.name,
        'SEQUENCE_TEMPLATE': junction.design_region,
        'SEQUENCE_PRIMER_PAIR_OK_REGION_LIST': [[1,junction.jmin_coordinate,junction.jmax_coordinate,junction.junction_length-junction.jmax_coordinate]]
    }

    #print(global_args)

    results = primer3.bindings.design_primers(
        seq_args=seq_args,
        global_args=global_args
    )

    # Generate tables
    tables = convert_primer_data_to_tables(results)

    results_list.append(results)

    # Display tables
    #for table_name, df in tables.items():
    #    print(f"\n=== {table_name.upper().replace('_', ' ')} ===")
    #    print(df.to_string())
    #    print()

    # Save to Excel with multiple sheets
    with pd.ExcelWriter('.out/' + junction.name + '_primer_analysis.xlsx', engine='openpyxl') as writer:
        for sheet_name, df in tables.items():
            df.to_excel(writer, sheet_name=sheet_name, index=True)

    # Save individual CSV files
    for table_name, df in tables.items():
        df.to_csv(f'.out/{junction.name}_primer_{table_name}.csv', index=True)

    print("Tables saved to:")
    print("- primer_analysis.xlsx (multi-sheet Excel file)")
    print("- Individual CSV files for each table")

Tables saved to:
- primer_analysis.xlsx (multi-sheet Excel file)
- Individual CSV files for each table
Tables saved to:
- primer_analysis.xlsx (multi-sheet Excel file)
- Individual CSV files for each table
Tables saved to:
- primer_analysis.xlsx (multi-sheet Excel file)
- Individual CSV files for each table


In [157]:
print(results_list[0])

{'PRIMER_LEFT_EXPLAIN': 'considered 5888, GC content failed 3, low tm 697, high tm 16, high hairpin stability 16, not in any ok left region 4968, ok 188', 'PRIMER_RIGHT_EXPLAIN': 'considered 5888, GC content failed 205, low tm 621, not in any ok right region 4968, ok 94', 'PRIMER_PAIR_EXPLAIN': 'considered 11, ok 11', 'PRIMER_LEFT_NUM_RETURNED': 10, 'PRIMER_RIGHT_NUM_RETURNED': 10, 'PRIMER_INTERNAL_NUM_RETURNED': 0, 'PRIMER_PAIR_NUM_RETURNED': 10, 'PRIMER_PAIR': [{'PENALTY': 7.027507843793611, 'COMPL_ANY_TH': 6.95568062357728, 'COMPL_END_TH': 6.216241422012217, 'PRODUCT_SIZE': 81, 'PRODUCT_TM': 78.0460981093121}, {'PENALTY': 7.34570487248286, 'COMPL_ANY_TH': 6.95568062357728, 'COMPL_END_TH': 6.216241422012217, 'PRODUCT_SIZE': 80, 'PRODUCT_TM': 78.20244588708988}, {'PENALTY': 7.34570487248286, 'COMPL_ANY_TH': 6.95568062357728, 'COMPL_END_TH': 6.216241422012217, 'PRODUCT_SIZE': 81, 'PRODUCT_TM': 78.0460981093121}, {'PENALTY': 7.470732076285573, 'COMPL_ANY_TH': 6.95568062357728, 'COMPL_EN

In [187]:
display(create_primer_dataframe(results_list[0]))

,pair_id,pair_penalty,product_size,product_tm,compl_any_th,compl_end_th,left_sequence,left_coords,left_tm,left_gc_percent,...,left_end_stability,right_sequence,right_coords,right_tm,right_gc_percent,right_penalty,right_self_any_th,right_self_end_th,right_hairpin_th,right_end_stability
0,0,10.244764,87,77.658920,6.955681,6.216241,TGAATTAGCTGTATCGTCAAGGCAC,"[163, 25]",58.841858,44.000000,...,5.01,AGGCCTGCTGAAAATGACTGA,"[249, 21]",57.358128,47.619048,3.784806,2.811118,0.0,0.0,3.41
1,1,10.292974,87,77.658920,6.955681,6.216241,TGAATTAGCTGTATCGTCAAGGCA,"[163, 24]",57.794196,41.666667,...,4.75,AGGCCTGCTGAAAATGACTGA,"[249, 21]",57.358128,47.619048,3.784806,2.811118,0.0,0.0,3.41
2,2,10.374586,89,77.851292,6.955681,6.216241,TCTGAATTAGCTGTATCGTCAAGGCA,"[161, 26]",59.707107,42.307692,...,4.75,AGGCCTGCTGAAAATGACTGA,"[249, 21]",57.358128,47.619048,3.784806,2.811118,0.0,0.0,3.41
3,3,10.469411,88,77.991212,6.955681,6.216241,CTGAATTAGCTGTATCGTCAAGGCA,"[162, 25]",58.615353,44.000000,...,4.75,AGGCCTGCTGAAAATGACTGA,"[249, 21]",57.358128,47.619048,3.784806,2.811118,0.0,0.0,3.41
4,4,10.510194,88,77.991212,6.955681,6.216241,CTGAATTAGCTGTATCGTCAAGGCAC,"[162, 26]",59.574214,46.153846,...,5.01,AGGCCTGCTGAAAATGACTGA,"[249, 21]",57.358128,47.619048,3.784806,2.811118,0.0,0.0,3.41
5,5,10.627848,86,77.799857,6.955681,6.216241,GAATTAGCTGTATCGTCAAGGCAC,"[164, 24]",57.463158,45.833333,...,5.01,AGGCCTGCTGAAAATGACTGA,"[249, 21]",57.358128,47.619048,3.784806,2.811118,0.0,0.0,3.41
6,6,10.679177,89,77.851292,1.035423,1.123604,TCTGAATTAGCTGTATCGTCAAGGC,"[161, 25]",58.387220,44.000000,...,4.35,AGGCCTGCTGAAAATGACTGA,"[249, 21]",57.358128,47.619048,3.784806,2.811118,0.0,0.0,3.41
7,7,10.691211,88,77.521185,6.955681,6.216241,TGAATTAGCTGTATCGTCAAGGCAC,"[163, 25]",58.841858,44.000000,...,5.01,AAGGCCTGCTGAAAATGACTGA,"[250, 22]",57.912558,45.454545,4.233970,5.394153,0.0,0.0,3.41
8,8,10.722865,87,77.658920,6.955681,0.000000,TGAATTAGCTGTATCGTCAAGGCAC,"[163, 25]",58.841858,44.000000,...,5.01,AGGCCTGCTGAAAATGACTGAA,"[249, 22]",57.912558,45.454545,4.231471,2.811118,0.0,0.0,3.02
9,9,10.739942,88,77.521185,6.955681,6.216241,TGAATTAGCTGTATCGTCAAGGCA,"[163, 24]",57.794196,41.666667,...,4.75,AAGGCCTGCTGAAAATGACTGA,"[250, 22]",57.912558,45.454545,4.233970,5.394153,0.0,0.0,3.41


In [ ]:
# Create the database
db_file = create_primer_database(results)

# Run some example queries
query_database(db_file)
    
print(f"\nDatabase '{db_file}' created successfully!")
print("You can now query it using any SQLite client or Python sqlite3.")